<a href="https://colab.research.google.com/github/krawch1989/iowa_liquor_sales/blob/stage/main_iowa_liquor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd